<a href="https://colab.research.google.com/github/JulienGremillot/OpenClassrooms_Ingenieur_Machine_Learning/blob/main/P7%20-%20D%C3%A9veloppez%20une%20preuve%20de%20concept/D%C3%A9veloppez_une_preuve_de_concept_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Fetch vision_transformer repository.
![ -d vision_transformer ] || git clone --depth=1 https://github.com/google-research/vision_transformer

Cloning into 'vision_transformer'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 37 (delta 4), reused 32 (delta 4), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [ ]:
# Install dependencies.
!pip install -qr vision_transformer/vit_jax/requirements.txt

     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 77 kB 7.0 MB/s 
     |████████████████████████████████| 207 kB 22.2 MB/s 
     |████████████████████████████████| 88 kB 9.8 MB/s 
     |████████████████████████████████| 179.8 MB 51 kB/s 
     |████████████████████████████████| 126 kB 82.3 MB/s 


In [ ]:
# Import files from repository.

import sys
if './vision_transformer' not in sys.path:
  sys.path.append('./vision_transformer')

%load_ext autoreload
%autoreload 2

from vit_jax import checkpoint
from vit_jax import models
from vit_jax import train
from vit_jax.configs import augreg as augreg_config
from vit_jax.configs import models as models_config

In [ ]:
# Some more imports used in this Colab.

import glob
import os
import random
import shutil
import time

from absl import logging
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

pd.options.display.max_colwidth = None
logging.set_verbosity(logging.INFO)  # Shows logs during training.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=False)
os.chdir("/content/gdrive/MyDrive/Colab Notebooks")
print("Répertoire de travail courant : %s" % os.getcwd())

Mounted at /content/gdrive
Répertoire de travail courant : /content/gdrive/MyDrive/Colab Notebooks


In [ ]:
# Le dataset "Stanford Dogs Dataset" offre une arborescence :
# images_12/{class_name}/{filename}.jpg
# on va faire un split physique pour des jeux "train" (90%) et "test" (10%) :
# images_12s/train/{class_name}/{filename}.jpg
# images_12s/test/{class_name}/{filename}.jpg

#base_dir = '/content/gdrive/MyDrive/Colab Notebooks/images_12'
dest_dir = '/content/gdrive/MyDrive/Colab Notebooks/images_12s'
#test_ratio = 0.1
#paths = glob.glob(f'{base_dir}/*/*.jpg')
#random.shuffle(paths)
#counts = dict(test=0, train=0)
#for i, path in enumerate(paths):
#    split = 'test' if i < test_ratio * len(paths) else 'train'
#    *_, class_name, basename = path.split('/')
#    dst = f'{dest_dir}/{split}/{class_name}/{basename}'
#    if not os.path.isdir(os.path.dirname(dst)):
#      os.makedirs(os.path.dirname(dst))
#    shutil.copy(path, dst)
#    counts[split] += 1

#print(f'Copié {counts["train"]:,} train and {counts["test"]:,} test images.')

In [ ]:
# Create a new temporary workdir.
workdir = f'./workdirs/{int(time.time())}'
workdir

'./workdirs/1638805968'

In [ ]:
# Get config for specified model.

# Note that we can specify simply the model name (in which case the recommended
# checkpoint for that model is taken), or it can be specified by its full
# name.
config = augreg_config.get_config('R_Ti_16')

# a vérifier ?
config.batch_eval = 120

# Some more parameters that you will often want to set manually.
# For example for VTAB we used steps={500, 2500} and lr={.001, .003, .01, .03}
config.base_lr = 0.01
config.shuffle_buffer = 1000
config.total_steps = 100
config.warmup_steps = 10
config.accum_steps = 0  # Not needed with R+Ti/16 model.
config.pp['crop'] = 224

# Read data from directory containing files.
config.dataset = dest_dir

In [ ]:
# Connect to TPUs if runtime type is of type TPU.

import os
if 'google.colab' in str(get_ipython()) and 'COLAB_TPU_ADDR' in os.environ:
  import jax
  import jax.tools.colab_tpu
  jax.tools.colab_tpu.setup_tpu()
  print('Connected to TPU.')
else:
  # Otherwise print information about GPU.
  !nvidia-smi

Mon Dec  6 15:52:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Call main training loop.
state = train.train_and_evaluate(config, workdir)

INFO:absl:Reading dataset from directories "/content/gdrive/MyDrive/Colab Notebooks/images_12s/train" and "/content/gdrive/MyDrive/Colab Notebooks/images_12s/test"
INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
INFO:absl:<PrefetchDataset shapes: {image: (1, 512, 224, 224, 3), label: (1, 512, 12)}, types: {image: tf.float32, label: tf.float32}>
INFO:absl:<PrefetchDataset shapes: {image: (1, 120, 224, 224, 3), label: (1, 120, 12)}, types: {image: tf.float32, label: tf.float32}>
INFO:absl:Selected fillename="R_Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0" for "R_Ti_16" with final_val=0.390
INFO:absl:Found no checkpoint at workdirs/1638805968
INFO:absl:Will start/continue training at initial_step=1
INFO:absl:Hyperparameters: {'accum_steps': 0, 'base_lr': 0.01, 'batch': 512, 'batch_eval': 120, 'checkpoint_every': 1000, 'd

ScopeParamShapeError: ignored

In [ ]:
# Load master table from Cloud.
with tf.io.gfile.GFile('gs://vit_models/augreg/index.csv') as f:
  df = pd.read_csv(f)

In [ ]:
# Let's first select the "best checkpoint" for every model. We show in the
# paper (section 4.5) that one can get a good performance by simply choosing the
# best model by final pre-train validation accuracy ("final-val" column).
# Pre-training with imagenet21k 300 epochs (ds=="i21k") gives the best
# performance in almost all cases (figure 6, table 5).
best_filenames = set(
    df.query('ds=="i21k"')
    .groupby('name')
    .apply(lambda df: df.sort_values('final_val').iloc[-1])
    .filename
)

# Select all finetunes from these models.
best_df = df.loc[df.filename.apply(lambda filename: filename in best_filenames)]

# Note: 9 * 68 == 612
len(best_filenames), len(best_df)

(10, 646)

In [ ]:
# More details for a single pre-trained checkpoint.
best_df.query('name=="R26+S/32" and adapt_ds=="stanford_dogs"')[[
  col for col in best_df.columns if col.startswith('adapt_')
]].sort_values('adapt_final_val')

,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,adapt_filename


In [ ]:
# Select a value from "adapt_filename" above that is a fine-tuned checkpoint.
filename = 'R26_S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_0.0-sd_0.0--oxford_iiit_pet-steps_0k-lr_0.003-res_384'

tfds_name = filename.split('--')[1].split('-')[0]
model_config = models_config.AUGREG_CONFIGS[filename.split('-')[0]]

In [ ]:
# Fetch dataset that the checkpoint was finetuned on.
# (Note that automatic download does not work with imagenet2012)
ds, ds_info = tfds.load('stanford_dogs', with_info=True)
ds_info

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: stanford_dogs/0.2.0
INFO:absl:Load dataset info from /tmp/tmpcrl5zv0utfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset stanford_dogs (/root/tensorflow_datasets/stanford_dogs/0.2.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

INFO:absl:Downloading http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar into /root/tensorflow_datasets/downloads/vision.stanfor.edu_aditya8_ImageNe_images_37w9QdJmofH4lgQ2uSGKteCsC7lFSZ5sEIEkraPsR4.tar.tmp.78a4a5db5994411d923adf2b101feaa2...


Dl Completed...: 0 url [00:00, ? url/s]

...

Shuffling and writing examples to /root/tensorflow_datasets/stanford_dogs/0.2.0.incompleteBTOUAJ/stanford_dogs-test.tfrecord


  0%|          | 0/8580 [00:00<?, ? examples/s]

INFO:absl:Done writing /root/tensorflow_datasets/stanford_dogs/0.2.0.incompleteBTOUAJ/stanford_dogs-test.tfrecord. Shard lengths: [2145, 2145, 2145, 2145]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/stanford_dogs/0.2.0


Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='stanford_dogs',
    version=0.2.0,
    description='The Stanford Dogs dataset contains images of 120 breeds of dogs from around
the world. This dataset has been built using images and annotation from
ImageNet for the task of fine-grained image categorization. There are
20,580 images, out of which 12,000 are used for training and 8580 for
testing. Class labels and bounding box annotations are provided
for all the 12,000 images.',
    homepage='http://vision.stanford.edu/aditya86/ImageNetDogs/main.html',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=120),
        'objects': Sequence({
            'bbox': BBoxFeature(shape=(4,), dtype=tf.float32),
        }),
    }),
    total_num_examples=20580,
    splits={
        'test': 8580,
        'train': 12000,
    },
    supervised_keys=('image'

In [ ]:
# Get model instance - no weights are initialized yet.
model = models.VisionTransformer(
    num_classes=ds_info.features['label'].num_classes, **model_config)

NameError: ignored